In [1]:
import os
import shutil
from ultralytics import YOLO
from tqdm.notebook import tqdm
import glob

print("Setup Complete. Libraries are imported.")

Setup Complete. Libraries are imported.


In [8]:
# Load the trained YOLO model
model = YOLO(MODEL_PATH)

# Get the mapping from class names to class IDs
class_names = model.names
print(f"Model Classes: {class_names}")

# 4. The exact name of the class you want to detection
TARGET_CLASS_NAME = 'bread-bag-opened'

# Find the specific class ID for our target class
try:
    # Create a reverse mapping from name to ID
    names_to_ids = {v: k for k, v in class_names.items()}
    target_class_id = names_to_ids[TARGET_CLASS_NAME]
    print(f"\nSuccessfully found target class: '{TARGET_CLASS_NAME}' with ID: {target_class_id}")
except KeyError:
    print(f"\nError: The class name '{TARGET_CLASS_NAME}' was not found in the model.")
    print("Please check the 'TARGET_CLASS_NAME' in Cell 2 and ensure it matches one of the model's classes.")
    target_class_id = -1  # Set to an invalid ID to stop execution

Model Classes: {0: 'bread-bag-closed', 1: 'bread-bag-opened'}

Successfully found target class: 'bread-bag-opened' with ID: 1


In [27]:
# --- CONFIGURATION ---

# 1. Path to your trained YOLO model
MODEL_PATH = 'data/model/detect_yolo_small_v3.pt'

# 2. Path to the folder containing all your image frames
#    (Supports .jpg, .jpeg, .png image formats)
SOURCE_FRAMES_PATH = 'D:\\Recordings\\New_Recordings\\Brown_Orange_Overlay_Frames'

# 3. Name of the folder where the filtered images will be saved
DESTINATION_PATH = 'D:\\Recordings\\New_Recordings\\Brown_Orange_Overlay_Frames\\Open'

# --- END OF CONFIGURATION ---


In [28]:
import os
import glob
import shutil
from tqdm import tqdm

from ultralytics import YOLO  # adjust if your YOLO import differs

# -------------------------------------------------------
# INPUTS
# -------------------------------------------------------
SOURCE_DIR = "/home/sunrise/BreadCounting/frames"     # your raw frames
MODEL_PATH = "/home/sunrise/BreadCounting/model.pt"    # YOLO model path
OUTPUT_ROOT = os.path.join(os.path.dirname(SOURCE_DIR), "sorted_by_class")
# -------------------------------------------------------

# Load YOLO model
model = YOLO(MODEL_PATH)

# Collect all image files
image_files = (
    glob.glob(os.path.join(SOURCE_DIR, "*.jpg")) +
    glob.glob(os.path.join(SOURCE_DIR, "*.jpeg")) +
    glob.glob(os.path.join(SOURCE_DIR, "*.png"))
)

print(f"Using model: {MODEL_PATH}")
print(f"Scanning source directory: {SOURCE_DIR}")
print(f"Found {len(image_files)} images")

# Create output root folder
os.makedirs(OUTPUT_ROOT, exist_ok=True)
print(f"Output root: {OUTPUT_ROOT}")

# Class name lookup from YOLO model
class_names = model.model.names

print("Detected YOLO classes:")
for cid, cname in class_names.items():
    print(f"  {cid}: {cname}")

print("\nRunning detection and sorting...")

# Process images
for image_path in tqdm(image_files):
    results = model(image_path, verbose=False)

    # A single image may return multiple 'results', but usually results[0] is enough
    result = results[0]

    # Extract detected class IDs
    detected_class_ids = result.boxes.cls.cpu().numpy().astype(int)

    # Skip images with no detections
    if len(detected_class_ids) == 0:
        continue

    # For each detected class → create folder → copy image
    for cid in set(detected_class_ids):
        class_name = class_names[cid]
        # clean folder name
        safe_class_name = class_name.replace(" ", "_").replace("/", "_")

        class_folder = os.path.join(OUTPUT_ROOT, safe_class_name)
        os.makedirs(class_folder, exist_ok=True)

        dst = os.path.join(class_folder, os.path.basename(image_path))
        shutil.copy2(image_path, dst)

print("\nCompleted.")
print(f"Sorted images are inside: {OUTPUT_ROOT}\n")


Model Path: data/model/best_detect.pt
Source Frames Path: D:\Recordings\New_Recordings\Brown_Orange_Overlay_Frames
Destination Path: D:\Recordings\New_Recordings\Brown_Orange_Overlay_Frames\Open

Found 750 images to process. Starting detection...


  0%|          | 0/750 [00:00<?, ?it/s]


Processing complete.
Copied 114 images containing 'bread-bag-opened' to the 'D:\Recordings\New_Recordings\Brown_Orange_Overlay_Frames\Open' folder.


In [7]:
if target_class_id != -1:
    # Get a list of all image files in the source directory
    image_files = glob.glob(os.path.join(SOURCE_FRAMES_PATH, '*.jpg')) + \
                  glob.glob(os.path.join(SOURCE_FRAMES_PATH, '*.jpeg')) + \
                  glob.glob(os.path.join(SOURCE_FRAMES_PATH, '*.png'))

    print(f"\nFound {len(image_files)} images to process. Starting detection...")

    images_copied = 0

    # Loop through each image with a progress bar
    for image_path in tqdm(image_files):
        # Run inference on the image
        results = model(image_path, verbose=False)  # verbose=False keeps the output clean

        # Check the results
        for result in results:
            # Get the class IDs of all detected objects in the current image
            detected_class_ids = result.boxes.cls.cpu().numpy().astype(int)

            # If our target class ID is in the list of detected IDs
            if target_class_id in detected_class_ids:
                # Copy the file to the destination folder
                file_name = os.path.basename(image_path)
                destination_file_path = os.path.join(DESTINATION_PATH, file_name)
                shutil.copy2(image_path, destination_file_path)
                images_copied += 1

                # We found the class, no need to check other detections in this image
                break

    print(f"\nProcessing complete.")
    print(f"Copied {images_copied} images containing '{TARGET_CLASS_NAME}' to the '{DESTINATION_PATH}' folder.")
else:
    print("\nSkipping image processing due to class name error.")# Load the trained YOLO model
model = YOLO(MODEL_PATH)

# Get the mapping from class names to class IDs
class_names = model.names
print(f"Model Classes: {class_names}")

# Find the specific class ID for our target class
try:
    # Create a reverse mapping from name to ID
    names_to_ids = {v: k for k, v in class_names.items()}
    target_class_id = names_to_ids[TARGET_CLASS_NAME]
    print(f"\nSuccessfully found target class: '{TARGET_CLASS_NAME}' with ID: {target_class_id}")
except KeyError:
    print(f"\nError: The class name '{TARGET_CLASS_NAME}' was not found in the model.")
    print("Please check the 'TARGET_CLASS_NAME' in Cell 2 and ensure it matches one of the model's classes.")
    target_class_id = -1 # Set to an invalid ID to stop execution

Model Classes: {0: 'bread-bag-closed', 1: 'bread-bag-opened'}

Successfully found target class: 'bread-bag-opened' with ID: 1
